# Week 02 Assignment glucose level data

Welcome to week two of this course programming 1. You will learn about time related data wrangling with pandas and you will learn to visualize with bokeh. This week will be focussed around missing data. Concretely, you will preprocess the glucose json file using interpolation to impute in order to conduct visual analysis. Learning outcomes:

- load a json dataset 
- typecast the Pandas DataFrame to appropiate data types
- inspect the dataset for quality and metadata information
- add a column with interpolated data in Pandas DataFrame
- perform visual analysis

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: prepare for inspection](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: interpolate the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: visualize the data](#4)
     - [Exercise 5.1](#ex-51)
- [part 6: Challenge](#5)
     - [Exercise 6.1](#ex-61)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import json

<a name='0'></a>
## Part 1: Load the data

Instructions: Load the json datafile `glucose.json` into a pandas dataframe. Check your dataframe with a `.head()` to compare with the expected outcome

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>json.load() method reads a file, pd.read_json converts it to a Pandas DataFrame</li>
    <li>when loading into a Pandas DataFrame use records orientation </li>
</ul>
</details>

<a name='ex-11'></a>
### Code your solution

In [2]:
# CODE YOUR SOLUTION HERE

# open en load json file
file = open("../data/glucose.json")
json_file = json.load(file)
file.close()

# Read json file in data frame
df_glucose = pd.read_json(json_file)
df_glucose.head()

,ID,time,recordtype,glucose
0,2845.0,2019-04-25 00:08,1,109
1,2850.0,2019-04-25 00:50,1,
2,2877.0,2019-04-25 07:02,1,123
3,2881.0,2019-04-25 07:34,1,158
4,2886.0,2019-04-25 08:19,1,


#### Expected outcome: 

<a name='1'></a>
## Part 2: Prepare the data

Check the datatypes of your dataframe. The `glucose` field should be an integer, the `time` field should have a datetime format. If the datatypes are different you should typecast them to the right format.
Make sure that your dataset is sorted by the time column


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use astype() method or pandas.DataFrame.to_datetime() for instance</li>
    <li>make sure that the empty spaces are filled with NaN. Use errors='coerce'</li>
    <li>set_index(), sort_index() and reset_index() are helpful to sort on index</li>
</ul>
</details>

<a name='ex-21'></a>
### Code your solution

In [3]:
df_glucose

,ID,time,recordtype,glucose
0,2.845000e+03,2019-04-25 00:08,1,109
1,2.850000e+03,2019-04-25 00:50,1,
2,2.877000e+03,2019-04-25 07:02,1,123
3,2.881000e+03,2019-04-25 07:34,1,158
4,2.886000e+03,2019-04-25 08:19,1,
...,...,...,...,...
131,1.614305e+19,2019-04-25 22:47,0,None
132,1.614305e+19,2019-04-25 23:02,0,None
133,1.614305e+19,2019-04-25 23:18,0,None
134,1.614305e+19,2019-04-25 23:33,0,None


In [4]:
# CODE YOUR SOLUTION HERE
df_glucose.sort_values("time", inplace = True)

df_glucose["ID"] = df_glucose["ID"].astype("int64")
df_glucose["time"] = df_glucose["time"].astype("datetime64")
df_glucose["glucose"] = pd.to_numeric(df_glucose["glucose"], errors = 'coerce').astype("float64")

df_glucose.dtypes

ID                     int64
time          datetime64[ns]
recordtype             int64
glucose              float64
dtype: object

#### Expected outcome: 

<a name='2'></a>
## Part 3: Inspect the data

Now that we prepared the data we are going to inspect the data to get more familiar with the data. You are required to do the following

- inspect the percentage missing data for glucose
- what is the relationship between recordtype and glucose value?
- what is the relationship between ID and glucose value?

Code the solutions to your answers. Create meaningful overviews or statistics

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>In the week 01 assignment some functions were explained to inspect missing values</li>
    <li>In the week 01 assignment some functions were explained to groupby value</li>
</ul>
</details>

<a name='ex-31'></a>
### Code your solution

In [15]:
#CODE YOUR SOLUTION HERE

n_missing_values = df_glucose["glucose"].isnull().sum()
n_total_values = df_glucose["glucose"].shape[0]
print(f"Percentage missing data: {n_missing_values / n_total_values}\n")

    
# Define the relationship between recordtype and glucose value by calculating the correlation between the two columns.
corr_recordtype_glucose = df_glucose[["recordtype", "glucose"]].corr()
print(f"Correlation recordtype ~ glucose: \n {corr_recordtype_glucose}")

# Define the relationship between ID and glucose value by calculating the correlation between the two columns.
corr_ID_glucose = df_glucose[["ID", "glucose"]].corr()
print(f"\nCorrelation ID ~ glucose: \n {corr_recordtype_glucose}")

# The correlation for recordtype ~ glucose, and ID ~ glucose cannot be calculated because the glucose column contains NaN values.

Percentage missing data: 0.6176470588235294

Correlation recordtype ~ glucose: 
             recordtype  glucose
recordtype         1.0      NaN
glucose            NaN      1.0

Correlation ID ~ glucose: 
             recordtype  glucose
recordtype         1.0      NaN
glucose            NaN      1.0


#### Expected outcome percentage missing data
0.6176470588235294

<a name='3'></a>
## Part 4: Interpolate the data

A lot of data is missing. Use interpolation to fill the missing values. Create a new column with the interpolated data. Take an argumentative approach. Select an interpolation method that suits the nature of the data and explain your choice. Mind you that the expected outcome of the interpolation values can differ from the example below

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use Pandas.DataFrame.interpolate() method</li>
</ul>
</details>

<a name='ex-41'></a>
### Code your solution

In [20]:
#CODE YOUR SOLUTION HERE
# Interpolate the missing data from the glucose column and assign it to a new column
df_glucose["interpolated"] = df_glucose['glucose'].interpolate(method = 'linear', limit_direction = 'forward')

df_glucose.head()

,ID,time,recordtype,glucose,interpolated
0,2845,2019-04-25 00:08:00,1,109.0,109.000000
54,-9223372036854775808,2019-04-25 00:14:00,0,NaN,109.466667
55,-9223372036854775808,2019-04-25 00:29:00,0,NaN,109.933333
56,-9223372036854775808,2019-04-25 00:44:00,0,NaN,110.400000
1,2850,2019-04-25 00:50:00,1,NaN,110.866667


Chose for the 'linear' method for interpolating, because it estimates missing values based on the increasing order of previous values. It also checks for the upcoming values. The 'limit_direction' arguments was set to forward, because the first value was known and can thus be used for interpolation. Limit was set to the default 0 because there are multiple NaN values in a row that we want to interpolate.  

#### Example outcome

<a name='4'></a>
## Part 5: Plot the data

Create a plot with the original data and the interpolated data. Consider what the best representation is for visualisation of actual values and modelled/imputed values. An example of such a plot is given below. This plot however is not considered the best practice. 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>figure(x_axis_type='datetime') automatically makes nices labels of the datetime data</li>
</ul>
</details>

<a name='ex-51'></a>
### Code your solution

In [33]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
output_notebook()

Loading BokehJS ...

In [32]:
#CODE YOUR SOLUTION HERE
p = figure(plot_width = 500, plot_height = 400, tools="pan, hover")

p.line(df_glucose['time'], df_glucose['interpolated'])
p.xaxis.axis_label = 'Date/Time (min)'
p.yaxis.axis_label = '(interpolated) Glucose levels (mmol/L)'


# output_notebook()
show(p)

<a name='6'></a>
## Part 6: Challenge

It might even be interesting to introduce a widget in which you can select different methods to interpolate.
1. Can you improve the interpolation by choosing an other method?
2. Can you add an rolling mean line? 
2. Can you improve the plot by making it interactive?

<a name='ex-61'></a>
### Code your solution

In [18]:
#CODE YOUR SOLUTION HERE